# IMDB 영화평 감성분석

In [6]:
import numpy as np 
import pandas as pd 

In [8]:
df = pd.read_csv('../00.data/corpus/labeledTrainData.tsv', sep='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [10]:
df.review.nunique()

24904

In [11]:

# 중복 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(24904, 3)

In [12]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [13]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />',' ')

In [14]:

# 영어 이외의 문자는 공백으로 변환
import re

df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]',' ',x))

In [15]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## Train/Test data set으로 만들기

In [16]:
from sklearn.model_selection import train_test_split

feature_df = df[['review']]
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size=0.3, stratify=df.sentiment, random_state=2021
)
X_train.shape, X_test.shape

((17432, 1), (7472, 1))

In [17]:
X_train.head(3)

,review
798,Beside the fact that in all it s awesomeness...
22217,In what would be his first screenplay based ...
8296,The second Care Bears movie is immensely bett...


## Feature Vectorization and Model Learning/Prediction/Evaluation

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- Count Vectorizer

In [19]:
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect.fit(X_train.review)
X_train_count = cvect.transform(X_train.review)
X_test_count = cvect.transform(X_test.review)

In [20]:
lr = LogisticRegression(C=10)
lr.fit(X_train_count, y_train)
pred = lr.predict(X_test_count)
accuracy_score(y_test, pred)

C:\Users\BSH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.887178800856531

- Tf-idf Vectorizer

In [21]:
tvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect.fit(X_train.review)
X_train_tfidf = tvect.transform(X_train.review)
X_test_tfidf = tvect.transform(X_test.review)

In [22]:

lr = LogisticRegression(C=10)
lr.fit(X_train_tfidf, y_train)
pred = lr.predict(X_test_tfidf)
accuracy_score(y_test, pred)

C:\Users\BSH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8957441113490364

## 학습한 모델을 저장하고 불러오기

In [25]:
import joblib

In [31]:
# 모델 저장
joblib.dump(tvect, 'model/imdb_tvect.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'model/imdb_tvect.pkl'

In [32]:

del tvect, lr

In [33]:
# 모델 불러오기
new_tvect = joblib.load('model/imdb_tvect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'model/imdb_tvect.pkl'

In [34]:
new_X_test = new_tvect.transform(X_test.review)

NameError: name 'new_tvect' is not defined

In [35]:

pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

NameError: name 'new_lr' is not defined

## Pipeline을 써서 학습/예측/평가

In [36]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

In [37]:
pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
accuracy_score(y_test, pred)

C:\Users\BSH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.887178800856531

## 최적의 하이퍼 파라미터 도출

In [39]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression())
])
params = ({
    'tfidf_vect__max_df': [100, 300, 500],
    'lr_clf__C': [1, 5, 10]
})

In [40]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf_vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr_clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'lr_clf__C': [1, 5, 10],
                         'tfidf_vect__max_df': [100, 300, 500]},
             scoring='accuracy', verbose=1)